# Benchmark Implementations of KNN

## Implementations

In [52]:
import numpy as np

np.random.seed(1337)
m, n = 30, 20
A = np.random.randn(m, 100)
B = np.random.randn(n, 100)

In [53]:
def three_loops(A, B):
    m, d = A.shape
    n = B.shape[0]
    dist = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            val = 0.0
            for k in range(d):
                val += (A[i][k] - B[j][k]) ** 2

            dist[i][j] = val
    return dist


C_3 = three_loops(A, B)

In [54]:
# This is how I would do in Matlab
def two_loops(A, B):
    m = A.shape[0]
    n = B.shape[0]
    dist = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            dist[i][j] = np.sum((A[i] - B[j]) ** 2)
    return dist


C_2 = two_loops(A, B)

In [55]:
def one_loop(A, B):
    m = A.shape[0]
    n = B.shape[0]
    dist = np.zeros((m, n))
    for i in range(m):
        dist[i, :] = np.sum((A[i] - B) ** 2, axis=-1)  # broadcasting A[i] to B

    return dist


"""
Alternative

def one_loop(A, B):
    m = A.shape[0]
    n = B.shape[0]
    dist = np.zeros((m, n))
    for j in range(n):
        dist[:, j] = np.sum((A - B[j]) ** 2, axis=-1)

    return dist
"""


C_1 = one_loop(A, B)

In [56]:
# matlab (no broadcasting => vector)
# image registration (ftt)
def zero_loop(A, B):
    A_square = np.sum(A**2, axis=-1, keepdims=True)  # (m, 1)
    B_square = np.sum(B**2, axis=-1, keepdims=True)  # (n, 1)

    A_mul_B = A @ B.T  # (m, d) @ (d, n) => (m, n)

    return A_square - 2 * A_mul_B + B_square.reshape(1, n)


C_0 = zero_loop(A, B)

In [57]:
assert np.allclose(C_3, C_2) and np.allclose(C_2, C_1) and np.allclose(C_1, C_0)

## Benchmark

m = 3, n = 2, d = 10
- 3 loops: 24.5e-6s
- 2 loops: 13e-6s
- 1 loop: 8.8e-6s
- 0 loop: 7.13e-6s

m = 30, n = 20, d = 100
- 3 loops: 22.9e-3s
- 2 loops: 1.31e-3s
- 1 loop: 139e-6s
- 0 loop: 14.3e-6s


In [105]:
%timeit -r 10 -n 2 three_loops(A, B)

22.9 ms ± 2.82 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [102]:

%timeit -r 100 -n 5 two_loop(A, B)

1.31 ms ± 220 µs per loop (mean ± std. dev. of 100 runs, 5 loops each)


In [90]:

%timeit -r 100 -n 2 one_loop(A, B)

139 µs ± 44.8 µs per loop (mean ± std. dev. of 100 runs, 2 loops each)


In [100]:

%timeit -r 1000 -n 5 zero_loop(A, B)

The slowest run took 12.83 times longer than the fastest. This could mean that an intermediate result is being cached.
14.3 µs ± 5.28 µs per loop (mean ± std. dev. of 1000 runs, 5 loops each)
